In [1]:
from codeflare_sdk import Cluster, ClusterConfiguration, TokenAuthentication

In [2]:
import os
import sys

In [3]:
# Authenticate the CodeFlare SDK
# On OpenShift, you can retrieve the token by running `oc whoami -t`,
# and the server with `oc cluster-info`.


auth = TokenAuthentication(
     token = "sha256~MZZVbtPDET4OZRmNYx065FSFw7FjO_O0cvTASbFQAl0",#os.getenv("AUTH_TOKEN"),
    server = "https://api.cluster-zbw6f.zbw6f.sandbox515.opentlc.com:6443",#os.getenv("API_SERVER"),
    skip_tls=True
)
auth.login()

Insecure request warnings have been disabled


'Logged into https://api.cluster-zbw6f.zbw6f.sandbox515.opentlc.com:6443'

In [4]:
# Configure the Ray cluster
cluster = Cluster(ClusterConfiguration(
    name='ray',
    namespace='llm-fine-tuning',
    num_workers=3,
    min_cpus=24,
    max_cpus=24,
    head_cpus=16,
    min_memory=32,
    max_memory=32,
    head_memory=32,
    head_gpus=1,
    num_gpus=1,
    image="quay.io/rhoai/ray:2.23.0-py39-cu121-2024.1",
    local_queue="local-queue-test",
))

Yaml resources loaded for ray


/opt/app-root/lib64/python3.11/site-packages/codeflare_sdk/ray/cluster/config.py:218: UserWarning: head_memory is being deprecated, use head_memory_requests and head_memory_limits
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/codeflare_sdk/ray/cluster/config.py:223: UserWarning: min_memory is being deprecated, use worker_memory_requests
  warnings.warn("min_memory is being deprecated, use worker_memory_requests")
/opt/app-root/lib64/python3.11/site-packages/codeflare_sdk/ray/cluster/config.py:226: UserWarning: max_memory is being deprecated, use worker_memory_limits
  warnings.warn("max_memory is being deprecated, use worker_memory_limits")
/opt/app-root/lib64/python3.11/site-packages/codeflare_sdk/ray/cluster/config.py:205: UserWarning: head_cpus is being deprecated, use head_cpu_requests and head_cpu_limits
  warnings.warn(
/opt/app-root/lib64/python3.11/site-packages/codeflare_sdk/ray/cluster/config.py:210: UserWarning: min_cpus is being deprecated, use worker_cpu_re

Output()

In [5]:
# Create the Ray cluster
cluster.up()

Ray Cluster: 'ray' has successfully been created


In [6]:
cluster.wait_ready()

Waiting for requested resources to be set up...
Requested cluster is up and running!
Dashboard is ready!


In [7]:
cluster.details()

                    🚀 CodeFlare Cluster Details 🚀                   
                                                                      
 ╭──────────────────────────────────────────────────────────────────╮ 
 │   Name                                                           │ 
 │   ray                                                Active ✅   │ 
 │                                                                  │ 
 │   URI: ray://ray-head-svc.llm-fine-tuning.svc:10001              │ 
 │                                                                  │ 
 │   ]8;id=530724;https://ray-dashboard-ray-llm-fine-tuning.apps.cluster-zbw6f.zbw6f.sandbox515.opentlc.com\Dashboard🔗]8;;\                                                    │ 
 │                                                                  │ 
 │                       Cluster Resources                          │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮      │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │      │ 
 │   │             │  │                                      │      │ 
 │   │  3          │  │  32G~32G     24~24       1           │      │ 
 │   │             │  │                                      │      │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯      │ 
 ╰──────────────────────────────────────────────────────────────────╯

RayCluster(name='ray', status=<RayClusterStatus.READY: 'ready'>, head_cpu_requests=16, head_cpu_limits=16, head_mem_requests='32G', head_mem_limits='32G', num_workers=3, worker_mem_requests='32G', worker_mem_limits='32G', worker_cpu_requests=24, worker_cpu_limits=24, namespace='llm-fine-tuning', dashboard='https://ray-dashboard-ray-llm-fine-tuning.apps.cluster-zbw6f.zbw6f.sandbox515.opentlc.com', worker_extended_resources={'nvidia.com/gpu': 1}, head_extended_resources={'nvidia.com/gpu': 1})

In [8]:
# Initialize the Job Submission Client
client = cluster.job_client

In [9]:
# Create the training and evaluation datasets.
# This can be run only once.
!{sys.executable} -m pip install datasets
import create_dataset
create_dataset.main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 10.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 304.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 268.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 359.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 340.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 351.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 169.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [10]:
# The S3 bucket where to store checkpoint.
# It can be set manually, otherwise it's retrieved from configured the data connection.
s3_bucket = ""
if not s3_bucket:
    s3_bucket = os.environ.get('AWS_S3_BUCKET')
assert s3_bucket, "An S3 bucket must be provided to store checkpoints"
print(s3_bucket)

models-afd583f2-380a-47da-9b1a-c43070ff3e09


In [11]:
submission_id = client.submit_job(
    entrypoint="python ray_finetuning_llm_deepspeed-online.py "
               "--model-name=meta-llama/Llama-2-7b-chat-hf "
               "--lora "
               "--num-devices=2 "
               "--num-epochs=3 "
               "--ds-config=./deepspeed_configs/zero_3_llama_2_7b.json "
               f"--storage-path={s3_bucket}/ray_finetune_llm_deepspeed/ "
               "--batch-size-per-device=16 "
               "--eval-batch-size-per-device=32 ",
    runtime_env={
        "env_vars": {
            "AWS_S3_ENDPOINT": os.environ.get('AWS_S3_ENDPOINT'),
            "AWS_ACCESS_KEY_ID": os.environ.get('AWS_ACCESS_KEY_ID'),
            "AWS_SECRET_ACCESS_KEY": os.environ.get('AWS_SECRET_ACCESS_KEY'),
            "AWS_DEFAULT_REGION": 'us-east-1',
            "AWS_S3_BUCKET": os.environ.get('AWS_S3_BUCKET')
        },
        "pip": "requirements.txt",
        "working_dir": "./",
        "excludes": ["/docs/", "*.ipynb", "*.md", "/tmp/"]
    },
)
print(submission_id)

2024-10-30 07:53:17,767	INFO dashboard_sdk.py:338 -- Uploading package gcs://_ray_pkg_ea6ee8edbebf577e.zip.
2024-10-30 07:53:17,768	INFO packaging.py:530 -- Creating a file package for local directory './'.


raysubmit_C1cgciaqX6CfsRjZ


In [20]:
client.stop_job(submission_id)

True

In [15]:
cluster.down()